<a href="https://colab.research.google.com/github/prateekguptaiiitk/Causal_Relation_Extraction/blob/master/Causal_Relation_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

crf_dataset.csv  crf_test_dataset.csv  sample_data


In [0]:
from google.colab import files
uploaded = files.upload()

Saving crf_dataset.csv to crf_dataset.csv


In [0]:
from google.colab import files
uploaded = files.upload()

Saving crf_test_dataset.csv to crf_test_dataset.csv


In [0]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('crf_dataset.csv', encoding='UTF-8')
test_data = pd.read_csv('crf_test_dataset.csv', encoding='UTF-8')

In [0]:
test_data = test_data.fillna(method='ffill')
train_data = train_data.fillna(method='ffill')

In [0]:
train_data.tail(10)

,Sentence #,Word,POS,Tag
25002,Sentence: 784,TRICHURIS,NNP,EE
25003,Sentence: 784,TRICHIURA,NNP,EE
25004,Sentence: 784,(,(,O
25005,Sentence: 784,whipworm,NN,O
25006,Sentence: 784,.,.,O
25007,Sentence: 785,1,CD,O
25008,Sentence: 785,",",",",O
25009,Sentence: 785,2,CD,O
25010,Sentence: 785,Cautions,NNS,O
25011,Sentence: 785,Contraindications,NNPS,O


In [0]:
train_words = list(set(train_data['Word'].values))
n_words = len(train_words)
print("No. of words in train data: ",n_words)            # no. of unique words in validation sentences

test_words = list(set(test_data['Word'].values))
n_words = len(test_words)
print("No. of words in test data: ",n_words)            # no. of unique words in validation sentences

No. of words in train data:  5892
No. of words in test data:  5062


In [0]:
class SentenceGetter(object):
  
  def __init__(self, data):
    self.n_sent = 1
    self.data = data
    self.empty = False
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]
    
  def get_next(self):
      try:
          s = self.grouped["Sentence: {}".format(self.n_sent)]
          self.n_sent += 1
          return s
      except:
          return None


In [0]:
getter1 = SentenceGetter(train_data)
train_sent = getter1.get_next()
print(train_sent)               # this is how a sentence looks now

getter2 = SentenceGetter(test_data)
test_sent = getter2.get_next()
print(test_sent)               # this is how a sentence looks now

[('Diagnosis', 'NNP', 'O'), ('specific', 'JJ', 'O'), ('malignancies', 'NNS', 'O'), ('available', 'JJ', 'O'), ('for', 'IN', 'O'), ('evaluation', 'NN', 'O'), ('included', 'VBD', 'O'), ('ALL', 'NNP', 'O'), (',', ',', 'O'), ('acute', 'JJ', 'O'), ('myeloid', 'NN', 'O'), ('leukaemia', 'NN', 'O'), ('(', '(', 'O'), ('AML', 'NNP', 'O'), (')', ')', 'O'), (',', ',', 'O'), ('Hodgkin', 'NNP', 'O'), ("'s", 'POS', 'O'), ('disease', 'NN', 'O'), (',', ',', 'O'), ('NHL', 'NNP', 'O'), (',', ',', 'O'), ('rhabdomyosarcoma', 'NN', 'O'), (',', ',', 'O'), ('neuroblastoma', 'NN', 'O'), (',', ',', 'O'), ('retinoblastoma', 'NN', 'O'), (',', ',', 'O'), ('OSTEOSARCOMA', 'NNP', 'CC'), ('Wilms', 'NNP', 'O'), ("'", 'POS', 'O'), ('tumour', 'NN', 'O'), (',', ',', 'O'), ('RETINOBLASTOMA', 'NNP', 'EE'), ('Ewings', 'NNP', 'O'), ("'", 'POS', 'O'), ('sarcoma', 'NN', 'O'), (',', ',', 'O'), ('central', 'JJ', 'O'), ('nervous', 'JJ', 'O'), ('system', 'NN', 'O'), ('(', '(', 'O'), ('CNS', 'NNP', 'O'), (')', ')', 'O'), ('tumours',

In [0]:
train_sentences = getter1.sentences
test_sentences = getter2.sentences

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_test = [sent2features(s) for s in test_sentences]
y_test = [sent2labels(s) for s in test_sentences]

In [0]:
!pip install sklearn_crfsuite
#Now we apply Conditional Random Field(CRF) algorithm
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

     |████████████████████████████████| 757kB 4.1MB/s 


In [0]:
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [0]:
from sklearn_crfsuite import metrics

y_pred = crf.predict(X_test)

print(metrics.flat_classification_report(
    y_test, y_pred, digits=3
))

metrics.flat_f1_score(y_test, y_pred,
                      average='weighted')

              precision    recall  f1-score   support

          CC      0.688     0.651     0.669      1183
          EE      0.652     0.560     0.602      1066
           O      0.975     0.989     0.982     14748
          RR      0.841     0.819     0.830       149

   micro avg      0.938     0.938     0.938     17146
   macro avg      0.789     0.755     0.771     17146
weighted avg      0.934     0.938     0.936     17146



0.9356230707925012